# JLCPCB Local Database Search

Requires parts database to already be downloaded and unzipped ```jlcpcb-components.sqlite3```

[Setup Python Virtual Environment in VSCode](/venv-setup.md)

In [ ]:
%pip install requests

In [1]:
import sqlite3
import json
import requests
import os


def download_file(url, filename):
    try:
        # Check if the file already exists
        if os.path.exists(filename):
            # Delete the existing file
            os.remove(filename)
            print(f"Deleted existing file: {filename}")

        response = requests.get(f"{url}/{filename}", stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(filename, "wb") as f:
            for chunk in response.iter_content(None):
                f.write(chunk)
        print(f"Downloaded {url} to {filename}")
    except requests.RequestException as e:
        print(f"Download {url} failed: {e}")


# URL of the SQLite database file
url = "https://cdfer.github.io/jlcpcb-parts-database"

# Local file path to save the database file
local_file = "jlcpcb-components.sqlite3"

# Download the file
download_file(url, local_file)

Deleted existing file: jlcpcb-components.sqlite3
Downloaded https://cdfer.github.io/jlcpcb-parts-database to jlcpcb-components.sqlite3


In [2]:
conn = sqlite3.connect(r"jlcpcb-components.sqlite3")
cur = conn.cursor()

cur.execute(
    "PRAGMA temp_store = MEMORY"
)  # Store temporary tables and indices in memory for faster access
cur.execute("PRAGMA mmap_size = 2147483648")  # Set the maximum memory map size to 2 GiB

In [11]:
import re

# Search term
search_term = "USB"

query = """
    SELECT c.price, c.stock, f.*
    FROM components c
    JOIN components_fts f ON c.lcsc = f.lcsc
    WHERE f.mfr LIKE ? OR f.description LIKE ? OR f.datasheet LIKE ? OR f.lcsc LIKE ?
"""

# Regex pattern to filter out LED but keep OLED
filter_pattern = re.compile(r"\bUSB\b", re.IGNORECASE)

rows = []

# Execute the query
cur.execute(
    query,
    (f"%{search_term}%",) * 4,
)

# Fetch the results and append to rows
rows = cur.fetchall()

# Filter rows using regex
filtered_rows = []
for row in rows:
    description = row[5]
    if filter_pattern.search(description):
        continue
    filtered_rows.append(row)

# Sort the rows based on the price
filtered_rows.sort(
    key=lambda row: json.loads(row[0])[0]["price"],
    reverse=True,
)

# Print the results
for i, row in enumerate(filtered_rows):
    price_data = json.loads(row[0])
    print(f"{row[3]}")
    print(f"Price: {round(price_data[0]['price'], 2)}USD")
    print(f"Stock: {row[1]}")
    print(f"Package: {row[4]}")
    print(f"Description: {row[5]}")
    print(f"https://jlcpcb.com/partdetail/C{row[2]}")
    print()

AT90USB1286-MU
Price: 17.32USD
Stock: 5
Package: QFN-64-EP(9x9)
Description: 128KB 2.7V~5.5V AVR 16MHz 48 QFN-64-EP(9x9)  Microcontrollers (MCU/MPU/SOC) ROHS
https://jlcpcb.com/partdetail/C614573

TUSB1046-DCIRNQR
Price: 14.88USD
Stock: 50
Package: WQFN-40-EP(4x6)
Description: QFN-40-EP(4x6)  Analog Switches / Multiplexers ROHS
https://jlcpcb.com/partdetail/C2151061

TUSB546AI-DCIRNQR
Price: 6.99USD
Stock: 30
Package: WQFN-40-EP(4x6)
Description: WQFN-40-EP(4x6)  Signal Buffers, Repeaters, Splitters ROHS
https://jlcpcb.com/partdetail/C702382

AT90USB162-16MU
Price: 5.58USD
Stock: 25
Package: QFN-32-EP(5x5)
Description: QFN-32-EP(5x5)  Microcontroller Units (MCUs/MPUs/SOCs) ROHS
https://jlcpcb.com/partdetail/C614212

AT90USB162-16AU
Price: 5.4USD
Stock: 5
Package: TQFP-32(7x7)
Description: 16KB 2.7V~5.5V AVR 16MHz 22 TQFP-32(7x7)  Microcontrollers (MCU/MPU/SOC) ROHS
https://jlcpcb.com/partdetail/C618722

USB83340AM-B-V02
Price: 3.99USD
Stock: 12
Package: VQFN-32-EP(5x5)
Description: VQF

In [17]:
# Close the database connection
conn.close()